Proprietary content. ©Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited

### Libraries along with their versions used at the time of making notebook-
google	2.0.3

nltk	3.2.5

numpy	1.18.1

pandas	0.25.3

tensorflow	2.1.0

Firstly, let's select TensorFlow version 2.x in colab

In [1]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

'2.4.0'

In [2]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

### Load the dataset

Read the data from the provided csv sms_spam_data.csv

In [5]:
# IMPORT DATA
import pandas as pd
import numpy as np

data = pd.read_csv('spam.csv', encoding='latin1')

Check info and shape of the data

In [6]:
print(data.shape)

(5572, 5)


In [7]:
print(data.info)

<bound method DataFrame.info of         v1  ... Unnamed: 4
0      ham  ...        NaN
1      ham  ...        NaN
2     spam  ...        NaN
3      ham  ...        NaN
4      ham  ...        NaN
...    ...  ...        ...
5567  spam  ...        NaN
5568   ham  ...        NaN
5569   ham  ...        NaN
5570   ham  ...        NaN
5571   ham  ...        NaN

[5572 rows x 5 columns]>


In [8]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [9]:
data.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [10]:
data.isnull().sum().sum()

16648

In [11]:
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

Drop irrelevant columns

In [14]:
data_clean = data.drop(columns={'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'})

In [15]:
data_clean.describe()

,v1,v2
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


Check frequency distribution of target column. Report whether the class is balanced or not?

In [16]:
data_clean.groupby('v1').count()

,v2
v1,
ham,4825
spam,747


Ans: Class is not balanced

In [18]:
dupes = data_clean.duplicated()
sum(dupes)

403

Create a new column having length of the text message and see it’s distribution for the two classes ( ham and spam) and comment your findings

In [27]:
data_clean['v2len'] = data_clean['v2'].str.len()

In [28]:
data_clean.head()

,v1,v2,v2len
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [30]:
data_clean.groupby('v1').mean()

,v2len
v1,
ham,71.023627
spam,138.866131


The lengths are also not balanced. Mean length for spam messages is more than for ham messages.

Encode the label column and make it numerical

In [35]:
data_input = data_clean.copy()

In [36]:
data_input.head()

,v1,v2,v2len
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [37]:
cleanup_nums = {"v1": {"ham": 0, "spam": 1}}

In [38]:
data_input = data_input.replace(cleanup_nums)
data_input.head()

,v1,v2,v2len
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


Preprocess the text data to make it suitable for model building

Convert the text message into lower case letters

In [39]:
data_input['lowerv2'] = data_input['v2'].str.lower()

In [40]:
data_input.head()

,v1,v2,v2len,lowerv2
0,0,"Go until jurong point, crazy.. Available only ...",111,"go until jurong point, crazy.. available only ..."
1,0,Ok lar... Joking wif u oni...,29,ok lar... joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,free entry in 2 a wkly comp to win fa cup fina...
3,0,U dun say so early hor... U c already then say...,49,u dun say so early hor... u c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro...",61,"nah i don't think he goes to usf, he lives aro..."


Remove stop words from the text message

In [45]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
from nltk.corpus import stopwords

In [47]:
stop = stopwords.words('english')

In [48]:
data_input['v2without_stopwords'] = data_input['lowerv2'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [49]:
data_input.head()

,v1,v2,v2len,lowerv2,v2without_stopwords
0,0,"Go until jurong point, crazy.. Available only ...",111,"go until jurong point, crazy.. available only ...","go jurong point, crazy.. available bugis n gre..."
1,0,Ok lar... Joking wif u oni...,29,ok lar... joking wif u oni...,ok lar... joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,49,u dun say so early hor... u c already then say...,u dun say early hor... u c already say...
4,0,"Nah I don't think he goes to usf, he lives aro...",61,"nah i don't think he goes to usf, he lives aro...","nah think goes usf, lives around though"


Remove punctuations

In [50]:
import string

In [53]:
data_input['v2final'] = data_input['v2without_stopwords'].str.replace('[{}]'.format(string.punctuation), '')

In [54]:
data_input.head()

,v1,v2,v2len,lowerv2,v2without_stopwords,v2final
0,0,"Go until jurong point, crazy.. Available only ...",111,"go until jurong point, crazy.. available only ...","go jurong point, crazy.. available bugis n gre...",go jurong point crazy available bugis n great ...
1,0,Ok lar... Joking wif u oni...,29,ok lar... joking wif u oni...,ok lar... joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,49,u dun say so early hor... u c already then say...,u dun say early hor... u c already say...,u dun say early hor u c already say
4,0,"Nah I don't think he goes to usf, he lives aro...",61,"nah i don't think he goes to usf, he lives aro...","nah think goes usf, lives around though",nah think goes usf lives around though


Segregate the data into X (independent feature - text message) and y (label) and split the data into train and test set ( take test size = 0.30)

In [55]:
X = data_input['v2final']

In [56]:
X.head()

0    go jurong point crazy available bugis n great ...
1                              ok lar joking wif u oni
2    free entry 2 wkly comp win fa cup final tkts 2...
3                  u dun say early hor u c already say
4               nah think goes usf lives around though
Name: v2final, dtype: object

In [57]:
y = data_input['v1']